In [1]:
from google.colab import files
data_to_load = files.upload()
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import time

Saving Game_of_Thrones_Script (1).csv to Game_of_Thrones_Script (1).csv


In [2]:
path = 'Game_of_Thrones_Script (1).csv'
script = pd.read_csv(path)

In [3]:
script = script.drop(['Release Date','Season','Episode','Episode Title'],axis=1)
script

,Name,Sentence
0,waymar royce,What do you expect? They're savages. One lot s...
1,will,I've never seen wildlings do a thing like this...
2,waymar royce,How close did you get?
3,will,Close as any man would.
4,gared,We should head back to the wall.
...,...,...
23906,brienne,I think we can all agree that ships take prece...
23907,bronn,I think that's a very presumptuous statement.
23908,tyrion lannister,I once brought a jackass and a honeycomb into ...
23909,man,The Queen in the North!


In [4]:
text = ''

for index, row in script.iterrows():
  add = str(row['Name']) + ' \n ' + str(row['Sentence']) + ' \n ' + ' \n '
  text += add


In [5]:
print(text[:495])

waymar royce 
 What do you expect? They're savages. One lot steals a goat from another lot and before you know it, they're ripping each other to pieces. 
  
 will 
 I've never seen wildlings do a thing like this. I've never seen a thing like this, not ever in my life. 
  
 waymar royce 
 How close did you get? 
  
 will 
 Close as any man would. 
  
 gared 
 We should head back to the wall. 
  
 royce 
 Do the dead frighten you? 
  
 gared 
 Our orders were to track the wildlings. We tracke


In [6]:
vocab = sorted(set(text))

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

w
a
y
m
a


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"waymar royce \n What do you expect? They're savages. One lot steals a goat from another lot and before"
" you know it, they're ripping each other to pieces. \n  \n will \n I've never seen wildlings do a thing "
"like this. I've never seen a thing like this, not ever in my life. \n  \n waymar royce \n How close did "
'you get? \n  \n will \n Close as any man would. \n  \n gared \n We should head back to the wall. \n  \n royce'
' \n Do the dead frighten you? \n  \n gared \n Our orders were to track the wildlings. We tracked them. Th'


In [10]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "waymar royce \n What do you expect? They're savages. One lot steals a goat from another lot and befor"
Target data: "aymar royce \n What do you expect? They're savages. One lot steals a goat from another lot and before"


In [12]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 74 ('w')
  expected output: 52 ('a')
Step    1
  input: 52 ('a')
  expected output: 76 ('y')
Step    2
  input: 76 ('y')
  expected output: 64 ('m')
Step    3
  input: 64 ('m')
  expected output: 52 ('a')
Step    4
  input: 52 ('a')
  expected output: 69 ('r')


In [13]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [14]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [16]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 90) # (batch_size, sequence_length, vocab_size)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23040     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 90)            92250     
Total params: 4,053,594
Trainable params: 4,053,594
Non-trainable params: 0
_________________________________________________________________


In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([87, 57, 66,  6,  4, 68, 20, 39, 53, 86, 85, 77, 55, 68, 21,  8, 83,
        3, 12, 60, 56, 15, 40, 66, 69, 82, 19, 85, 58,  8, 16,  3, 74, 30,
       18,  2, 75, 16, 31, 50, 53, 17, 33, 22,  9, 59,  0, 87, 18, 58, 57,
       87, 17, 39, 13,  6, 46, 82, 25,  9, 60, 58, 40,  3, 46,  6, 36, 26,
       13, 36,  6,  1, 10, 29, 16, 47, 64, 57, 55, 79, 19, 48, 87, 48, 87,
       22, 39, 48, 33, 33, 33, 47, 53, 21, 55,  8, 52, 67, 65, 82])

In [20]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'was born in Flea Bottom. \n  \n melisandre \n Your blood is noble. \n  \n gendry baratheon \n Are you sayi'

Next Char Predictions: 
 '“fo*#q9Pb‘—zdq;-ū"1ie4Qorē8—g-5"wG7!x5H[b6J>.h\n“7gf“6P2*WēB.igQ"W*MC2M* /F5Xmfdé8Y“Y“>PYJJJXb;d-apnē'


In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 90)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.500821


In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
checkpoint_dir = './GOT_RNN_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
EPOCHS = 40
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
301/301 [==============================] - 16s 54ms/step - loss: 2.2131
Epoch 2/40
301/301 [==============================] - 17s 55ms/step - loss: 1.4205
Epoch 3/40
301/301 [==============================] - 17s 56ms/step - loss: 1.2017
Epoch 4/40
301/301 [==============================] - 17s 57ms/step - loss: 1.1135
Epoch 5/40
301/301 [==============================] - 18s 58ms/step - loss: 1.0625
Epoch 6/40
301/301 [==============================] - 18s 60ms/step - loss: 1.0262
Epoch 7/40
301/301 [==============================] - 18s 60ms/step - loss: 0.9975
Epoch 8/40
301/301 [==============================] - 18s 60ms/step - loss: 0.9730
Epoch 9/40
301/301 [==============================] - 18s 59ms/step - loss: 0.9498
Epoch 10/40
301/301 [==============================] - 18s 60ms/step - loss: 0.9288
Epoch 11/40
301/301 [==============================] - 18s 60ms/step - loss: 0.9076
Epoch 12/40
301/301 [==============================] - 18s 60ms/step - loss: 0.8877
E

In [25]:
tf.train.latest_checkpoint(checkpoint_dir)

'./GOT_RNN_training_checkpoints/ckpt_40'

In [26]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [27]:
def generate_text(model, start_string):
  num_generate = 10000

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
print(generate_text(model, start_string=u"tyrion lannister "))

tyrion lannister 
 In ferticing you now? Why? Why did you tell him to do with that again? 
  
 jon snow 
 We've got to warm of work away. 
  
 tormund 
 We'red in the night. 
  
 qyburn 
 Surely there is one of those times. 
  
 arya stark 
 Stop! Please. 
  
 jorah mormont 
 Make me. 
  
 sam 
 We are shed being shoot? 
  
 varys 
 That is a fight. We're fighting them. We all live and burn the rest. You're my dear... 
  
 sansa stark 
 What in sthen she was writing in the wind? One of a war is waiting for you. 
  
 sansa stark 
 He's a beauty one. 
  
 jaime lannister 
 The queen's orpannister 
 I thought he is my family home Tommen, if she steppid them. 
  
 and very good, all the same armor been. What are you doing here? 
  
 benjen Stormborn old innocence. I was sold that you could be something for each of them. When will you stayed back to your boots like it? 
  
 illyrio 
 If they do that why are you doing, Cleg where a drop of penjention of dans and slaughter even forgive. 
  
 